# gate_preprocessing 을 검증하기 위한 프로그램

In [1]:
# 각 크리스탈의 중심값을 얻어내기위한 함수 (3x3)
# find_xyz_center(디텍터 갯수, 크리스탈의 x축 갯수, 크리스탈의 y축 갯수, 크리스탈의 z축 갯수, 원점으로부터의 거리, 크리스탈의 x축 길이, 크리스탈의 y축 길이, 크리스탈의 z축 길이)
def YSS_find_xyz_center(detector_amount, crystal_x_amount, crystal_y_amount, crystal_z_amount, r, x_len, y_len, z_len,):
    crystal_x_num = int((int(crystal_x_amount)/2))
    
    
    result = []
    for i in range(int(detector_amount)):
        detector = []
        crystal = []
        crystal_z_num = int((int(crystal_z_amount)/2))
        for j in range(int(crystal_z_amount)):
            
            crystal_y_num = int((int(crystal_y_amount)/2))
            for k in range(int(crystal_y_amount)):
                crystal.append([r + (x_len/2), (y_len * crystal_y_num), (z_len * crystal_z_num)])
                crystal_y_num = crystal_y_num - 1
            crystal_z_num = crystal_z_num - 1
        detector.append(crystal)
        result.append(detector)
    result = list(itertools.chain(*result))
    detector_rot = result
    result = []
    
    detector_num = 0
    for i in range(int(detector_amount)):
        det_rot = []
        for j in range(len(detector_rot[i])):
            det_rot.append([format((detector_rot[i][j][0]*math.cos(((360/detector_amount)*detector_num)*math.pi/180)) - (detector_rot[i][j][1]*math.sin(((360/detector_amount)*detector_num)*math.pi/180)),".3f"),\
                           format((detector_rot[i][j][0]*math.sin(((360/detector_amount)*detector_num)*math.pi/180)) + (detector_rot[i][j][1]*math.cos(((360/detector_amount)*detector_num)*math.pi/180)),".3f"),\
                           format(detector_rot[i][j][2],".3f")])
        detector_num += 1
        result.append(det_rot)
    
            
    
    return result

In [2]:
# 각 디텍터의 중심값을 받아와 크리스탈의 모서리의 좌표를 구하는 함수
def find_crystal_edge(x, y, z, x_len, y_len, z_len):
    x = float(x)
    y = float(y)
    z = float(z)
    x_len = float(x_len)
    y_len = float(y_len)
    z_len = float(z_len)
    edge_a1 = [x-(x_len/2), y+(y_len/2), z-(z_len/2)]
    edge_b1 = [x-(x_len/2), y+(y_len/2), z+(z_len/2)]
    edge_c1 = [x-(x_len/2), y-(y_len/2), z-(z_len/2)]
    edge_d1 = [x-(x_len/2), y-(y_len/2), z+(z_len/2)]
    edge_a2 = [x+(x_len/2), y+(y_len/2), z-(z_len/2)]
    edge_b2 = [x+(x_len/2), y+(y_len/2), z+(z_len/2)]
    edge_c2 = [x+(x_len/2), y-(y_len/2), z-(z_len/2)]
    edge_d2 = [x+(x_len/2), y-(y_len/2), z+(z_len/2)]
    
    
    crystal_edge = [edge_a1, edge_b1, edge_c1, edge_d1, edge_a2, edge_b2, edge_c2, edge_d2]
    #print(crystal_edge)
    return crystal_edge

In [3]:
# 데이터 범위를 디텍터 갯수에 맞게 미리 나누어내는 방법
# find_detector(원본 데이터, 모서리값, 디텍터 갯수)


def find_detector(rotate_crystal_edge_chunks):
    for i in range(len(rotate_crystal_edge_chunks)):
        x_max = -1000
        x_min = 1000
        y_max = -1000
        y_min = 1000
        z_max = -1000
        z_min = 1000
        for j in range(len(rotate_crystal_edge_chunks[i])):
            if(float(rotate_crystal_edge_chunks[i][j][0]) > float(x_max)):
                x_max = rotate_crystal_edge_chunks[i][j][0]
            if(float(rotate_crystal_edge_chunks[i][j][0]) < float(x_min)):
                x_min = rotate_crystal_edge_chunks[i][j][0]
            if(float(rotate_crystal_edge_chunks[i][j][1]) > float(y_max)):
                y_max = rotate_crystal_edge_chunks[i][j][1]
            if(float(rotate_crystal_edge_chunks[i][j][1]) < float(y_min)):
                y_min = rotate_crystal_edge_chunks[i][j][1]
            if(float(rotate_crystal_edge_chunks[i][j][2]) > float(z_max)):
                z_max = rotate_crystal_edge_chunks[i][j][2]
            if(float(rotate_crystal_edge_chunks[i][j][2]) < float(z_min)):
                z_min = rotate_crystal_edge_chunks[i][j][2]
        print(str(i)+" : " + str(x_min) + " < x < " + str(x_max) + ", " + str(y_min) + " < y < " + str(y_max)\
             + ", " + str(z_min) + " < z < " + str(z_max))
        

# 검증 시작

In [4]:
import os, glob, csv, codecs
import shutil
import re
import itertools
import math

## 실제 데이터의 첫번째 coincidence 좌표 사용

In [5]:
# 실제 데이터의 1번째 coincidence 좌표
test_data = [[-7.885, 45.56, -0.002718], [8.186, -45.07, -0.4635]]

In [6]:
# test_data_single1 은 2번 디텍터 헤드에 위치해 있기 때문에 회전 각도를 120도로 설정

# 먼저 데이터를 0번 디텍터 위치로 회전변환을 해야하므로 각도는 360-120 deg 하여 이동시킴
rotate_test_data_single1 = ([(test_data[0][0]*math.cos((360-120)*math.pi/180))-(test_data[0][1]*math.sin((360-120)*math.pi/180)),\
                    (test_data[0][0]*math.sin((360-120)*math.pi/180))+(test_data[0][1]*math.cos((360-120)*math.pi/180)),\
                    test_data[0][2]])
print("---------test_data_single1 이 회전된 좌표---------")
print(rotate_test_data_single1)
print("---------------------------------------------")

# 이제 0번 디텍터에서 몇번 크리스탈에 속하는지 알아내야 한다.


---------test_data_single1 이 회전된 좌표---------
[43.39861739641902, -15.951389691159726, -0.002718]
---------------------------------------------


In [7]:
# 각 크리스탈의 중심값 찾기 (detector_num, crystal_x_amount, crystal_y_amount, crystal_z_amount, r길이, x_len, y_len, z_len)

crystal_location = YSS_find_xyz_center(6, 1, 3, 3, 44, 3, 11, 13)

print(crystal_location[0])

[['45.500', '11.000', '13.000'], ['45.500', '0.000', '13.000'], ['45.500', '-11.000', '13.000'], ['45.500', '11.000', '0.000'], ['45.500', '0.000', '0.000'], ['45.500', '-11.000', '0.000'], ['45.500', '11.000', '-13.000'], ['45.500', '0.000', '-13.000'], ['45.500', '-11.000', '-13.000']]


In [8]:
# 각 크리스탈의 모서리 값 구하기
origin_crystal_edge = []
for j in range(len(crystal_location[0])):
    # find_crystal_edge(크리스탈 중점 x좌표, 크리스탈 중점 y좌표, 크리스탈 중점 z좌표, 디텍터의 x_len, y_len z_len)
    origin_crystal_edge.append(find_crystal_edge(crystal_location[0][j][0],\
                          crystal_location[0][j][1],\
                          crystal_location[0][j][2],\
                          3, 11, 13))  
    
#print(origin_crystal_edge[8])

### 0번 디텍터의 각 크리스탈 범위

In [9]:
print("---------test_data_single1 이 회전된 좌표---------")
print(rotate_test_data_single1)
print("---------------------------------------------")
detector_number = find_detector(origin_crystal_edge)

---------test_data_single1 이 회전된 좌표---------
[43.39861739641902, -15.951389691159726, -0.002718]
---------------------------------------------
0 : 44.0 < x < 47.0, 5.5 < y < 16.5, 6.5 < z < 19.5
1 : 44.0 < x < 47.0, -5.5 < y < 5.5, 6.5 < z < 19.5
2 : 44.0 < x < 47.0, -16.5 < y < -5.5, 6.5 < z < 19.5
3 : 44.0 < x < 47.0, 5.5 < y < 16.5, -6.5 < z < 6.5
4 : 44.0 < x < 47.0, -5.5 < y < 5.5, -6.5 < z < 6.5
5 : 44.0 < x < 47.0, -16.5 < y < -5.5, -6.5 < z < 6.5
6 : 44.0 < x < 47.0, 5.5 < y < 16.5, -19.5 < z < -6.5
7 : 44.0 < x < 47.0, -5.5 < y < 5.5, -19.5 < z < -6.5
8 : 44.0 < x < 47.0, -16.5 < y < -5.5, -19.5 < z < -6.5


### single1 의 데이터는 5번 크리스탈에 해당하는것을 확인
    따라서 rotate_test_data_single1 값을 0번디텍터 헤드의 5번 크리스탈 중심값으로 변경

In [10]:
rotate_test_data_single1 = crystal_location[0][5]
print(rotate_test_data_single1)

['45.500', '-11.000', '0.000']


## 크리스탈 위치로 변경했으니 다시 원위치로 회전변환

In [11]:
# test_data_single1 은 2번 디텍터 헤드에 위치해 있기 때문에 회전 각도를 120도로 설정

final_test_data_single1 = ([(float(rotate_test_data_single1[0])*math.cos((120)*math.pi/180))-(float(rotate_test_data_single1[1])*math.sin((120)*math.pi/180)),\
                    (float(rotate_test_data_single1[0])*math.sin((120)*math.pi/180))+(float(rotate_test_data_single1[1])*math.cos((120)*math.pi/180)),\
                    float(rotate_test_data_single1[2])])
print("---------test_data_single1 의 초기 좌표---------")
print(test_data[0])
print("---------------------------------------------")
print("---------test_data_single1 의 최종 좌표---------")
print(final_test_data_single1)
print("---------------------------------------------")

# 이제 0번 디텍터에서 몇번 크리스탈에 속하는지 알아내야 한다.


---------test_data_single1 의 초기 좌표---------
[-7.885, 45.56, -0.002718]
---------------------------------------------
---------test_data_single1 의 최종 좌표---------
[-13.223720558371163, 44.904155872191964, 0.0]
---------------------------------------------


## single2 에도 같은 작업을 실행

In [12]:
# single2 의 최초 데이터
test_data[1]

[8.186, -45.07, -0.4635]

In [13]:
# test_data_single1 은 5번 디텍터 헤드에 위치해 있기 때문에 회전 각도를 300도로 설정

# 먼저 데이터를 0번 디텍터 위치로 회전변환을 해야하므로 각도는 360-300 deg 하여 이동시킴
rotate_test_data_single2 = ([(test_data[1][0]*math.cos((360-300)*math.pi/180))-(test_data[1][1]*math.sin((360-300)*math.pi/180)),\
                    (test_data[1][0]*math.sin((360-300)*math.pi/180))+(test_data[1][1]*math.cos((360-300)*math.pi/180)),\
                    test_data[1][2]])
print("---------test_data_single2 이 회전된 좌표---------")
print(rotate_test_data_single2)
print("---------------------------------------------")

# 이제 0번 디텍터에서 몇번 크리스탈에 속하는지 알아내야 한다.


---------test_data_single2 이 회전된 좌표---------
[43.12476494856465, -15.445716044620589, -0.4635]
---------------------------------------------


In [14]:
print("---------test_data_single2 이 회전된 좌표---------")
print(rotate_test_data_single2)
print("---------------------------------------------")
detector_number = find_detector(origin_crystal_edge)

---------test_data_single2 이 회전된 좌표---------
[43.12476494856465, -15.445716044620589, -0.4635]
---------------------------------------------
0 : 44.0 < x < 47.0, 5.5 < y < 16.5, 6.5 < z < 19.5
1 : 44.0 < x < 47.0, -5.5 < y < 5.5, 6.5 < z < 19.5
2 : 44.0 < x < 47.0, -16.5 < y < -5.5, 6.5 < z < 19.5
3 : 44.0 < x < 47.0, 5.5 < y < 16.5, -6.5 < z < 6.5
4 : 44.0 < x < 47.0, -5.5 < y < 5.5, -6.5 < z < 6.5
5 : 44.0 < x < 47.0, -16.5 < y < -5.5, -6.5 < z < 6.5
6 : 44.0 < x < 47.0, 5.5 < y < 16.5, -19.5 < z < -6.5
7 : 44.0 < x < 47.0, -5.5 < y < 5.5, -19.5 < z < -6.5
8 : 44.0 < x < 47.0, -16.5 < y < -5.5, -19.5 < z < -6.5


## single1 과 똑같이 5번 크리스탈 범위에 들어가는것을 확인

In [15]:
rotate_test_data_single2 = crystal_location[0][5]
print(rotate_test_data_single2)

['45.500', '-11.000', '0.000']


In [16]:
# test_data_single1 은 5번 디텍터 헤드에 위치해 있기 때문에 회전 각도를 300도로 설정

final_test_data_single2 = ([(float(rotate_test_data_single2[0])*math.cos((300)*math.pi/180))-(float(rotate_test_data_single2[1])*math.sin((300)*math.pi/180)),\
                    (float(rotate_test_data_single2[0])*math.sin((300)*math.pi/180))+(float(rotate_test_data_single2[1])*math.cos((300)*math.pi/180)),\
                    float(rotate_test_data_single2[2])])
print("---------test_data_single12 의 초기 좌표---------")
print(test_data[1])
print("---------------------------------------------")
print("---------test_data_single12 의 최종 좌표---------")
print(final_test_data_single2)
print("---------------------------------------------")

# 이제 0번 디텍터에서 몇번 크리스탈에 속하는지 알아내야 한다.


---------test_data_single12 의 초기 좌표---------
[8.186, -45.07, -0.4635]
---------------------------------------------
---------test_data_single12 의 최종 좌표---------
[13.223720558371175, -44.90415587219196, 0.0]
---------------------------------------------
